In [1]:
%reset -f
import pandas as pd
import requests
import json

In [2]:
# API endpoints
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
player_history_url = 'https://fantasy.premierleague.com/api/element-summary/{player_id}/'

In [3]:
response = requests.get(url)

if response.status_code == 200:
    print('The API request was successful.')
else:
    print(f'Error: The API request failed with status code {response.status_code}.')

The API request was successful.


In [4]:
pd.options.display.max_rows = 999

In [5]:
response = requests.get(url)
players = response.json()['elements']

In [6]:
element_types = response.json()['element_types']
position_mapping = {}

for pos in element_types:
    position_mapping[pos['id']] = pos['singular_name']

In [7]:
# Extract past season info
player_info = []

print("Please wait a few minutes...")

for player in players:
    player_id = player['id']
    name = player['first_name'] + ' ' + player['second_name']
    position = position_mapping[player['element_type']]
    value = player['now_cost']/10
  
    history_response = requests.get(player_history_url.format(player_id=player_id))
    points = 0

    try:
        history_data = history_response.json()['history_past']
        for season in history_data:
            if season['season_name'] == "2022/23":
                points = season['total_points']
            else: 
                pass
    except (KeyError, IndexError):
        print(f"Error: No history for {player_id}: {name} found in response")
        points = 0
  
    player_info.append({'name': name,'position': position, 'value': value, 'points': points})

df = pd.DataFrame(player_info)

Please wait a few minutes...


In [8]:
# Points per million
df['ppm'] = round(df['points'] / df['value'],2)
df = df.sort_values('ppm', ascending=False)

df

,name,position,value,points,ppm
490,Kieran Trippier,Defender,5.0,198,39.60
652,José Malheiro de Sá,Goalkeeper,4.5,148,32.89
482,Nick Pope,Goalkeeper,5.0,157,31.40
581,Granit Xhaka,Midfielder,5.0,153,30.60
0,David Raya Martín,Goalkeeper,5.5,166,30.18
376,Alisson Becker,Goalkeeper,5.5,162,29.45
463,Marcus Rashford,Midfielder,7.0,205,29.29
41,Tyrone Mings,Defender,4.5,130,28.89
513,Aaron Ramsdale,Goalkeeper,5.0,143,28.60
189,Pervis Estupiñán Tenorio,Defender,4.5,128,28.44


In [9]:
pos_only = df[df['position'] == 'Defender']
pos_only = pos_only.sort_values('points', ascending=False)

pos_only

,name,position,value,points,ppm
490,Kieran Trippier,Defender,5.0,198,39.60
10,Benjamin White,Defender,5.5,156,28.36
4,Gabriel dos Santos Magalhães,Defender,6.0,146,24.33
486,Fabian Schär,Defender,5.5,139,25.27
41,Tyrone Mings,Defender,4.5,130,28.89
488,Dan Burn,Defender,5.0,129,25.80
487,Sven Botman,Defender,5.0,129,25.80
189,Pervis Estupiñán Tenorio,Defender,4.5,128,28.44
383,Virgil van Dijk,Defender,6.0,127,21.17
455,Luke Shaw,Defender,4.5,124,27.56
